# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

In [268]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

# 8-1 TF-IDF 활용 핵심키워드 추출

In [269]:
import MeCab

t = MeCab.Tagger()

## 실습 1. sklearn 활용


In [270]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')

  cont_tag = soup.select('div._article_body_contents')
  text = cont_tag[0].text.strip()
  
  return text 


docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

In [271]:
docs

['// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 

### 1) 전처리

In [324]:
from konlpy.tag import Mecab
mecab = Mecab()

text = mecab.pos(docs[0])
text[0][0]

'과기'

In [325]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_doc = []
  tagged_doc = mecab.pos(doc)
  for word in tagged_doc:
    if 'NNG'in word:
      preprocessed_doc.append(word[0])
    elif 'NNP' in word:
      preprocessed_doc.append(word[0])
    elif 'NNB' in word:
      preprocessed_doc.append(word[0])
    elif 'VV' in word:
      preprocessed_doc.append(word[0])
  preprocessed_docs.append(preprocessed_doc)      

preprocessed_docs[1][:100]

['한국전력공사',
 '한국전력공사',
 '제공',
 '나주',
 '연합뉴스',
 '송',
 '기자',
 '한국전력',
 '전력',
 '데이터',
 '활용',
 '신',
 '新',
 '서비스',
 '개발',
 '경진',
 '대회',
 '한전',
 '아트',
 '센터',
 '개최',
 '산업',
 '통상',
 '자원',
 '부',
 '주최',
 '한전',
 '전력',
 '그룹',
 '광주',
 '과학',
 '기술원',
 '주관',
 '이번',
 '행사',
 '전력',
 '빅',
 '데이터',
 '활용',
 '사업',
 '모델',
 '발굴',
 '마련',
 '대회',
 '참가',
 '희망',
 '학생',
 '일반인',
 '벤처',
 '기업',
 '새싹',
 '기업',
 '경진',
 '대회',
 '홈페이지',
 '제안서',
 '내',
 '서류',
 '심사',
 '통과',
 '팀',
 '개인',
 '제공',
 '전력',
 '데이터',
 '활용',
 '서비스',
 '개발',
 '데이터',
 '외부',
 '반출',
 '불가',
 '경우',
 '한전',
 '아트',
 '센터',
 '마련',
 '데이터',
 '공유',
 '센터',
 '개발',
 '완성도',
 '기술',
 '상용',
 '가능',
 '확장',
 '등',
 '기준',
 '사내',
 '직원',
 '사외',
 '전문',
 '위원',
 '평가',
 '시상',
 '개인',
 '학생',
 '일반인',
 '기업',
 '부문']

In [326]:
preprocessed_docs_flat = []
for doc in preprocessed_docs:
  flat = ','.join(doc)
  preprocessed_docs_flat.append(flat)
preprocessed_docs_flat

['과기,정통부,유영민,장관,등,참석,기념행사,투입,여종,데이터,구축,민간,클라우드,외부,연계,체계,개방,강화,데일리,이재운,기자,국가,차원,빅,데이터,활용,시대,산업,창출,기존,산업,변화,이르,혁신,장,센터,문,분야,데이터,경제,발전,정부,청사진,현실,구현,데,계획,과학,기술,정보,통신부,서울,중구,대한,상공,회의소,데이터,생태,조성,혁신,성장,기반,마련,빅,데이터,플랫,폼,센터,출범식,행사,개최,유영민,과기,정통부,장관,노웅래,국회,과학,기술,정보,방송,통신,위원회,위원장,등,참가,분야,센터,간,투입,이미지,픽사,베이,빅,데이터,데이터,활용,혁신,장,이루,문재,정부,경제,성장,핵심,요소,중,문재인,대통령,들,데이터,활용,정보,보호,보안,중요,강조,맥락,속,빅,데이터,센터,공공,민간,협업,활용,양질,데이터,생산,구축,플랫,폼,수집,분석,유통,역할,담당,과기,정통부,분야,플랫,폼,연계,기관,센터,구축,데,투입,계획,올해,규모,사업,추진,대상,분야,금융,카드,환경,한국,수자원,공사,문화,한국,문화,정보원,교통,한국,교통,연구원,헬,스케어,국립암센터,유통,소비,매일,방송,통신,중소기업,비즈,지역,경제,경기도,청,산림,한국,임업,흥원,등,공모,빅,데이터,센터,선정,다음,달,공모,추가,선정,지원,운영,계획,데이터,생태계,혁신,기업,경쟁력,제고,역할,수행,주요,활용,전략,사례,보,빅,데이터,활용,신,新,시장,창출,방안,담,금융,플랫,폼,경우,소,상공,신용,평가,고도,등,금융,취약,계층,대상,중,금리,대출,절감,연간,신규,대출,창출,전망,유통,소비,중소기업,플랫,폼,소상,공인,중소기업,폐업,감소,문화,플랫,폼,문화,예술,관람,생활,체육,참여,높이,방안,모색,의료비,절감,헬스,케어,기업,매출,향상,산업,육성,통신,산림,등,눈길,과기,정통부,제공,여종,데이터,구축,알고리즘,제공,센터,분야,데이터,부족,문제,해소,올해,말,시장,수요,여종,신규,데이터,생산,구축,사업,완료,여종,양질,데이터,생산,구축,시장,공급,계획,공공,민간,사이,데이터,파일,형식,등,호환,문제,해소,개방,표

### 2) TF-IDF 계산

In [327]:
from sklearn.feature_extraction.text import CountVectorizer

#BoW 생성
count_vect = CountVectorizer()
BoW = count_vect.fit_transform(preprocessed_docs_flat)
BoW


<5x569 sparse matrix of type '<class 'numpy.int64'>'
	with 742 stored elements in Compressed Sparse Row format>

In [328]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_trans = TfidfTransformer()
tfidf= tfidf_trans.fit_transform(BoW)
tfidf.todense()
#tfidf 생성

matrix([[0.        , 0.01527764, 0.        , ..., 0.        , 0.03206183,
         0.        ],
        [0.        , 0.0349748 , 0.        , ..., 0.        , 0.        ,
         0.07339856],
        [0.02712719, 0.01292625, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.01466522, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.03610061, 0.01894029, ..., 0.03788059, 0.        ,
         0.        ]])

### 3) 핵심키워드 추출

In [329]:
import pandas as pd
vocab = count_vect.get_feature_names()
df = pd.DataFrame(tfidf.todense(), columns = vocab)
df = df.transpose()
df

,0,1,2,3,4
가공,0.000000,0.000000,0.027127,0.000000,0.000000
가능,0.015278,0.034975,0.012926,0.014665,0.036101
가동,0.000000,0.000000,0.000000,0.000000,0.018940
가운데,0.000000,0.000000,0.000000,0.000000,0.018940
가입자,0.000000,0.000000,0.000000,0.030777,0.000000
...,...,...,...,...,...
회의,0.000000,0.000000,0.000000,0.000000,0.037881
회의소,0.032062,0.000000,0.000000,0.000000,0.000000
효율,0.000000,0.000000,0.000000,0.000000,0.037881
흥원,0.032062,0.000000,0.000000,0.000000,0.000000


In [330]:
tf =tfidf.toarray()
tf

array([[0.        , 0.01527764, 0.        , ..., 0.        , 0.03206183,
        0.        ],
       [0.        , 0.0349748 , 0.        , ..., 0.        , 0.        ,
        0.07339856],
       [0.02712719, 0.01292625, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01466522, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03610061, 0.01894029, ..., 0.03788059, 0.        ,
        0.        ]])

In [331]:
sorted_docs = []

for doc in tf:
  idx_sorted = doc.argsort()
  doc_sorted = doc[idx_sorted[::-1]]
  sorted_docs.append(doc_sorted)

sorted_docs

[array([0.47360671, 0.22443284, 0.192371  , 0.16805399, 0.16030917,
        0.16030917, 0.13749872, 0.12933643, 0.12933643, 0.12933643,
        0.1288331 , 0.12824734, 0.12824734, 0.12824734, 0.12824734,
        0.10736092, 0.10736092, 0.10346914, 0.10346914, 0.0961855 ,
        0.0961855 , 0.0961855 , 0.0961855 , 0.0961855 , 0.0961855 ,
        0.0961855 , 0.0961855 , 0.0961855 , 0.0961855 , 0.08588874,
        0.08588874, 0.07760186, 0.07760186, 0.07760186, 0.07760186,
        0.07760186, 0.06412367, 0.06412367, 0.06412367, 0.06412367,
        0.06412367, 0.06412367, 0.06412367, 0.06412367, 0.06412367,
        0.06412367, 0.06412367, 0.06412367, 0.06412367, 0.06412367,
        0.06412367, 0.06412367, 0.06412367, 0.06412367, 0.06412367,
        0.06412367, 0.06412367, 0.06412367, 0.05418923, 0.05173457,
        0.05173457, 0.05173457, 0.05173457, 0.05173457, 0.05173457,
        0.05173457, 0.05173457, 0.05173457, 0.05173457, 0.04294437,
        0.04294437, 0.03612616, 0.03612616, 0.03

In [332]:
def sort_keywords(keywords):
    #키워드 sort
  arr =keywords.toarray().flatten()
  sort_idx = arr.argsort()[::-1]

  return sort_idx 

In [333]:
k =sort_keywords(tfidf[0])
k

array([126, 517,  39, 246,  49, 411, 560, 544, 196,  34,  62, 228, 297,
       175, 162, 413,  83, 526, 143, 303, 499, 333, 460, 380, 434, 127,
       495, 465, 289, 407, 166,  17,  73, 195, 335, 274, 428,  33,  59,
       288, 412, 476, 316, 174, 484, 276,  43, 403, 469, 218, 533, 118,
       249, 358, 241, 555, 120, 216, 217, 558, 320,  56, 300, 230, 160,
        50,  80, 259,  88, 326, 522,  81, 235, 473,  41, 208, 409, 374,
       310,  78, 158, 306, 159, 437, 123, 361, 122,  72, 155, 442, 129,
       357, 313, 150, 328,  96, 330, 325, 324, 101,  93, 149, 322, 337,
       342, 104, 151, 424, 105, 348, 152, 418, 349, 351,  87, 109, 110,
       401, 317, 315, 301, 212, 370,  38, 553, 184, 161, 263, 509, 262,
       186, 514, 260, 518, 257, 520, 552, 551,  27, 254, 525, 251,  24,
       250, 192, 231,  19, 539,  15, 202,  14, 543, 545, 268, 502, 121,
       468,  68,  66, 452, 163, 454,  63, 565, 458, 459,  61, 462, 463,
       290, 466,  12,  40, 168, 474, 567, 211, 206,  54, 481, 48

In [334]:
extract_keywords(vocab, k)

['데이터', '플랫', '계획', '센터', '과기']

In [335]:
def extract_keywords(feature_names, sorted_keywords, n=5):
    #키워드 추출
    result = []
    for i in range(n):
      result.append(feature_names[sorted_keywords[i]])

    return result

In [336]:
from sklearn.feature_extraction.text import TfidfTransformer


doc = preprocessed_docs_flat[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_trans.transform(count_vect.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
 
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
데이터
플랫
계획
센터
과기


----

쌤 ver

In [337]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ")
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return content.strip()
  


docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
print(docs[0])

과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.주요 활용 전략·사례를 보면 빅데이터 활용을 통

1) 전처리

In [347]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  word_list = (mecab.pos(doc))
  token_list =[]
  for word_info in word_list:
    if word_info[1][0] == 'N' or word_info[1] == 'VV':
      token_list.append(word_info[0])
  
  # print(" ".join(token_list))
  preprocessed_docs.append( " ".join(token_list))
  
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 산업 '

In [348]:
## 리스트 컴프리헨션으로 축약
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] == "N" or token[1] == 'VV']))

print(len(preprocessed_docs))
preprocessed_docs[4][:100]

5


'금융 결제 원 신용 정보원 등 지정 비씨 카드 추가 포함 여부 논의 시행령 발표 후 추진 하반기 데이터 유통 판매 급물살 정부 금융 결제 원 신용 정보원 등 업 곳 데이터 중계 기'

2) TF-IDF 계산

In [349]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
DTM = count_vect.fit_transform(preprocessed_docs)
list(count_vect.vocabulary_.keys())[:10]

['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [350]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True)
tfidf_transformer.fit(DTM)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

3) 핵심 키워드 추출

In [363]:
def sort_keywords(keywords):
  return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
    
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    #키워드 추출
    extract_keywords = []
    for idx, score in sorted_keywords[:n]:
      extract_keywords.append((feature_names[idx], score))
    return extract_keywords

In [367]:
# 리스트 컴프리헨션으로 만들기 
def sort_keywords(keywords):
  return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
  
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    #키워드 추출

    return [(feature_names[idx], score) for (idx, score) in sorted_keywords[:n]]


In [368]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names() # TF-IDF 단어 목록
# print(feature_names)
tf_idf_vector = tfidf_transformer.transform(count_vect.transform([doc])) # 문서의 tf-idf 추출
# print(tf_idf_vector)
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
print(sorted_keywords)
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
print(keywords)
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
# for k in keywords:
    # print(k)

[(518, 0.2623579798457269), (39, 0.2248782684391945), (410, 0.1873985570326621), (49, 0.1873985570326621), (546, 0.15119197585668623), (196, 0.15119197585668623), (34, 0.15119197585668623), (62, 0.15060359722931801), (296, 0.14991884562612967), (228, 0.14991884562612967), (175, 0.14991884562612967), (162, 0.14991884562612967), (412, 0.12550299769109835), (83, 0.12550299769109835), (528, 0.12095358068534899), (143, 0.12095358068534899), (500, 0.11243913421959725), (496, 0.11243913421959725), (466, 0.11243913421959725), (460, 0.11243913421959725), (434, 0.11243913421959725), (379, 0.11243913421959725), (332, 0.11243913421959725), (302, 0.11243913421959725), (288, 0.11243913421959725), (127, 0.11243913421959725), (406, 0.10040239815287867), (166, 0.10040239815287867), (334, 0.09071518551401174), (273, 0.09071518551401174), (195, 0.09071518551401174), (73, 0.09071518551401174), (17, 0.09071518551401174), (557, 0.07495942281306484), (535, 0.07495942281306484), (485, 0.07495942281306484), (4

In [369]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names() # TF-IDF 단어 목록
# print(feature_names)
tf_idf_vector = tfidf_transformer.transform(count_vect.transform([doc])) # 문서의 tf-idf 추출
# print(tf_idf_vector)
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.2623579798457269)
('계획', 0.2248782684391945)
('정통부', 0.1873985570326621)
('과기', 0.1873985570326621)
('혁신', 0.15119197585668623)
